In [1]:
import sys
import pathlib as pl 
tmp = pl.Path(".").absolute() / "code" / "pymicro"
sys.path.append(str(tmp))
print(sys.path)

['/home/nathanrapin/cours_info/Projet_SO3_quaternions/Quaternion', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/nathanrapin/python-env/lib/python3.8/site-packages', '/home/nathanrapin/cours_info/Projet_SO3_quaternions/Quaternion/code/pymicro']


In [2]:
from pymicro.crystal.quaternion import Quaternion

In [3]:
import math
import numpy as np

n = 11
eulers = []
phi1 = np.linspace(0.+0.00001, np.pi-0.00001, n, endpoint=True)
Phi = phi1.copy()
phi2 = phi1.copy()
for i in range(len(phi1)):
    for j in range(len(Phi)):
        for k in range(len(phi2)):
            eulers.append([phi1[i], Phi[j], phi2[k]])

eulers = np.array(eulers)
print(eulers)
print(len(eulers))

[[1.00000000e-05 1.00000000e-05 1.00000000e-05]
 [1.00000000e-05 1.00000000e-05 3.14167265e-01]
 [1.00000000e-05 1.00000000e-05 6.28324531e-01]
 ...
 [3.14158265e+00 3.14158265e+00 2.51326812e+00]
 [3.14158265e+00 3.14158265e+00 2.82742539e+00]
 [3.14158265e+00 3.14158265e+00 3.14158265e+00]]
1331


In [4]:
epsilon = np.finfo('float').eps
epsilon

2.220446049250313e-16

In [5]:
def upper_hemishpere_axis(axis):
    if abs(axis[2]) > epsilon:
        if axis[2] < 0.:
            # reverse axis
            return -1. * axis
    elif abs(axis[1]) > epsilon:
        if axis[1] < 0:
            # reverse axis and zero z component
            return np.array([-axis[0], -axis[1], 0.])
    elif axis[0] < 0.:
        # reverse axis and zero y and z components
        return np.array([-axis[0], 0., 0.])
    return axis

def eu2ro_vect(euler):
    """
    Compute the rodrigues vector from the 3 euler angles (in radians).
    :param euler: array of the 3 Euler angles (in radians).
    :return: array of the rodrigues vector as a 3 components numpy array.
    """
    a = 0.5 * (euler[:, 0] - euler[:, 2])
    b = 0.5 * (euler[:, 0] + euler[:, 2])
    r1 = np.tan(0.5 * euler[:, 1]) * np.cos(a) / np.cos(b)
    r2 = np.tan(0.5 * euler[:, 1]) * np.sin(a) / np.cos(b)
    r3 = np.tan(b)
    return np.column_stack((r1, r2, r3))

def eu2om_vect(euler):
    """
    Compute the orientation matrix from the 3 euler angles (in radians).
    :param euler: array of the 3 Euler angles (in radians).
    :return: array of the orientation matrices
    """
    c1 = np.cos(euler[:, 0])
    s1 = np.sin(euler[:, 0])
    c = np.cos(euler[:, 1])
    s = np.sin(euler[:, 1])
    c2 = np.cos(euler[:, 2])
    s2 = np.sin(euler[:, 2])
    # rotation matrix g
    g11 = c1 * c2 - s1 * s2 * c
    g12 = s1 * c2 + c1 * s2 * c
    g13 = s2 * s
    g21 = -c1 * s2 - s1 * c2 * c
    g22 = -s1 * s2 + c1 * c2 * c
    g23 = c2 * s
    g31 = s1 * s
    g32 = -c1 * s
    g33 = c
    g = np.column_stack([g11[:], g12[:], g13[:], g21[:], g22[:], g23[:], g31[:], g32[:], g33[:]]).reshape((len(euler), 3, 3))
    return g

def eu2qu_vect(euler):
    """
    Compute the quaternion from the 3 euler angles (in radians).
    :param euler: array of the 3 euler angles in radians.
    :return: array of `Quaternion` instances representing the rotation.
    """
    P = -1
    phi1 = euler[:, 0]
    Phi = euler[:, 1]
    phi2 = euler[:, 2]
    q0 = np.cos(0.5 * (phi1 + phi2)) * np.cos(0.5 * Phi)
    q1 = np.cos(0.5 * (phi1 - phi2)) * np.sin(0.5 * Phi)
    q2 = np.sin(0.5 * (phi1 - phi2)) * np.sin(0.5 * Phi)
    q3 = np.sin(0.5 * (phi1 + phi2)) * np.cos(0.5 * Phi)
    q = Quaternion([q0, -P * q1, -P * q2, -P * q3], convention=P)
    if q0 < 0:
        # the scalar part must be positive
        q.quat = q.quat * -1
    # ambiguous rotation
    if q.quat[0] < 3 * epsilon:
        axis = upper_hemishpere_axis(q.quat[1:])
        q.quat = np.array([0., *axis])
    return q.quat

def eu2ax(euler):
    """
    Compute the (axis, angle) representation associated to this (passive)
    rotation expressed by the Euler angles.
    :param euler: 3 euler angles (in radians).
    :returns: a tuple containing the axis (a vector) and the angle (in radians).
    """
    t = np.tan(0.5 * euler[1])
    s = 0.5 * (euler[0] + euler[2])
    d = 0.5 * (euler[0] - euler[2])
    tau = np.sqrt(t ** 2 + np.sin(s) ** 2)
    alpha = 2 * np.arctan2(tau, np.cos(s))
    if alpha > np.pi:
        axis = np.array([-t / tau * np.cos(d), -t / tau * np.sin(d), -1 / tau * np.sin(s)])
        angle = 2 * np.pi - alpha
    else:
        axis = np.array([t / tau * np.cos(d), t / tau * np.sin(d), 1 / tau * np.sin(s)])
        angle = alpha
    return np.array([*axis, angle])

def ax2om(ax):
    """
    Compute the (passive) orientation matrix associated the rotation
    defined by the given (axis, angle) pair.

    :param axis: the rotation axis.
    :param angle: the rotation angle (radians).
    :returns: the 3x3 orientation matrix.
    """
    c = np.cos(ax[3])
    s = np.sin(ax[3])
    g = np.array([[c + (1 - c) * ax[0] ** 2,
                   (1 - c) * ax[0] * ax[1] + s * ax[2],
                   (1 - c) * ax[0] * ax[2] - s * ax[1]],
                  [(1 - c) * ax[0] * ax[1] - s * ax[2],
                   c + (1 - c) * ax[1] ** 2,
                   (1 - c) * ax[1] * ax[2] + s * ax[0]],
                  [(1 - c) * ax[0] * ax[2] + s * ax[1],
                   (1 - c) * ax[1] * ax[2] - s * ax[0],
                   c + (1 - c) * ax[2] ** 2]])
    return g

def ax2eu(ax):
    return om2eu(ax2om(ax))

def ax2ro(ax):
    if abs(ax[3] - np.pi) < epsilon:
        # handle this case
        pass
    return ax[:3] * np.tan(ax[3] / 2)
    
def ax2qu(ax):
    """
    Compute the quaternions associated to the the rotations defined by the given
    (axis, angle) pair.

    :param ax: array of 4 components-vector composed by the rotation axis 
        and the rotation angle (radians).
    :return: the corresponding Quaternions.
    """
    half_angle = 0.5 * ax[:, 3]
    sin_half_angle = np.sin(half_angle)
    quat = np.empty((ax.shape[0], 4))
    quat[:, 0] = np.cos(half_angle)
    quat[:, 1:] = sin_half_angle[:, np.newaxis] * ax[:, :3]
    return quat

def om2eu(g):
    """
    Compute the Euler angles from the orientation matrix.

    This conversion follows the paper of Rowenhorst et al. :cite:`Rowenhorst2015`.
    In particular when :math:`g_{33} = 1` within the machine precision,
    there is no way to determine the values of :math:`\phi_1` and :math:`\phi_2`
    (only their sum is defined). The convention is to attribute
    the entire angle to :math:`\phi_1` and set :math:`\phi_2` to zero.

    :param g: The 3x3 orientation matrix
    :return: The 3 euler angles in radians.
    """
    eps = np.finfo('float').eps
    (phi1, Phi, phi2) = (0.0, 0.0, 0.0)
    # treat special case where g[2, 2] = 1
    if np.abs(g[2, 2]) >= 1 - eps:
        if g[2, 2] > 0.0:
            phi1 = np.arctan2(g[0][1], g[0][0])
        else:
            phi1 = -np.arctan2(-g[0][1], g[0][0])
            Phi = np.pi
    else:
        Phi = np.arccos(g[2][2])
        zeta = 1.0 / np.sqrt(1.0 - g[2][2] ** 2)
        phi1 = np.arctan2(g[2][0] * zeta, -g[2][1] * zeta)
        phi2 = np.arctan2(g[0][2] * zeta, g[1][2] * zeta)
    # ensure angles are in the range [0, 2*pi]
    if phi1 < 0.0:
        phi1 += 2 * np.pi
    if Phi < 0.0:
        Phi += 2 * np.pi
    if phi2 < 0.0:
        phi2 += 2 * np.pi
    return np.array([phi1, Phi, phi2])

def om2ax(om):
    P = -1  # q.convention
    diag_delta = -P * np.array([om[1, 2] - om[2, 1], om[2, 0] - om[0, 2], om[0, 1] - om[1, 0]])
    # make sure cos(omega) is within [-1, 1]
    t = np.clip(0.5 * (np.trace(om) - 1), -1.0, 1.0)
    omega = np.arccos(t)
    if omega < 2 * epsilon:
        return np.array([0.0, 0.0, 1.0, 0.0])
    # determine the right eigenvector corresponding to the eigenvalue of +1
    w, v = np.linalg.eig(om)
    axis = np.real(v.T[np.isclose(w, 1.0 + 0.0j)])[0]
    # check signs, including when diag delta terms are zeros
    axis = np.where(np.abs(diag_delta) < 1e-12, axis, np.abs(axis) * np.sign(diag_delta))
    return np.array([*axis, omega])

def om2ro(om):
    return eu2ro(om2eu(om))

def om2qu(om):
    # code inspired from 
    with np.errstate(invalid='ignore', divide='ignore'):
        trace = om[0][0] + om[1][1] + om[2][2]
        s = [
             0.5 / np.sqrt(1.0 + trace),
             2.0 * np.sqrt(1.0 + om[0, 0] - om[1, 1] - om[2, 2]),
             2.0 * np.sqrt(1.0 + om[1, 1] - om[2, 2] - om[0, 0]),
             2.0 * np.sqrt(1.0 + om[2, 2] - om[0, 0] - om[1, 1])
             ]
        if trace > 0:
            q0 = 0.25 / s[0]
            q1 = -(om[2, 1] - om[1, 2] ) * s[0]
            q2 = -(om[0, 2] - om[2, 0] ) * s[0]
            q3 = -(om[1, 0] - om[0, 1] ) * s[0]
        elif om[0,0] > np.maximum(om[1, 1], om[2, 2]):
            q0 = (om[2, 1] - om[1, 2]) / s[1]
            q1 = -0.25 * s[1]
            q2 = -(om[0, 1] + om[1, 0]) / s[1]
            q3 = -(om[0, 2] + om[2, 0]) / s[1]
        elif om[1,1] > om[2,2]:
            q0 = (om[0, 2] - om[2, 0]) / s[2]
            q1 = -(om[0, 1] + om[1, 0]) / s[2]
            q2 = -0.25 * s[2]
            q3 = -(om[1, 2] + om[2, 1]) / s[2]
        else:
            q0 = (om[1, 0] - om[0, 1]) / s[3]
            q1 = -(om[0, 2] + om[2, 0]) / s[3]
            q2 = -(om[1, 2] + om[2, 1]) / s[3]
            q3 = -0.25 * s[3]
    q = np.array([q0, q1, q2, q3])
    if q[0] < 0.:
        q *= -1.
    return q

def ro2om(rod):
    """
    Compute the orientation matrix from the Rodrigues vector.

    :param rod: The Rodrigues vector as a 3 components array.
    :returns: The 3x3 orientation matrix representing the rotation.
    """
    r = np.linalg.norm(rod)
    I = np.eye(3)
    if r < epsilon:
        # the rodrigues vector is zero, return the identity matrix
        return I
    theta = 2 * np.arctan(r)
    n = rod / r
    omega = np.array([[0.0, n[2], -n[1]],
                      [-n[2], 0.0, n[0]],
                      [n[1], -n[0], 0.0]])
    g = I + np.sin(theta) * omega + (1 - np.cos(theta)) * omega.dot(omega)
    return g

def ro2eu(rod):
    return om2eu(ro2om(rod))

def ro2ax(rod):
    """
    Compute the axis/angle representation from the Rodrigues vector.

    :param rod: The Rodrigues vector as a 3 components array.
    :returns: A tuple in the (axis, angle) form.
    """
    r = np.linalg.norm(rod)
    axis = rod / r
    angle = 2 * np.arctan(r)
    return np.array([*axis, angle])

def ro2qu(rod):
    return ax2qu(ro2ax(rod))

def qu2eu(q):
    P = -1
    (q0, q1, q2, q3) = q
    q03 = q0 ** 2 + q3 ** 2
    q12 = q1 ** 2 + q2 ** 2
    chi = np.sqrt(q03 * q12)
    if chi == 0.:
        if q12 == 0.:
            phi_1 = np.arctan2(-2 * P * q0 * q3, q0 ** 2 - q3 ** 2)
            Phi = 0.
        else:
            phi_1 = np.arctan2(-2 * q1 * q2, q1 ** 2 - q2 ** 2)
            Phi = pi
        phi_2 = 0.
    else:
        phi_1 = np.arctan2((q1 * q3 - P * q0 * q2) / chi,
                      (-P * q0 * q1 - q2 * q3) / chi)
        Phi = np.arctan2(2 * chi, q03 - q12)
        phi_2 = np.arctan2((P * q0 * q2 + q1 * q3) / chi,
                      (q2 * q3 - P * q0 * q1) / chi)
    return np.array([phi_1, Phi, phi_2])
    
def qu2om(q):
    P = -1  # q.convention
    (q0, q1, q2, q3) = q
    qbar = q0 ** 2 - q1 ** 2 - q2 ** 2 - q3 ** 2
    g = np.array([[qbar + 2 * q1 ** 2, 2 * (q1 * q2 - P * q0 * q3), 2 * (q1 * q3 + P * q0 * q2)],
                  [2 * (q1 * q2 + P * q0 * q3), qbar + 2 * q2 ** 2, 2 * (q2 * q3 - P * q0 * q1)],
                  [2 * (q1 * q3 - P * q0 * q2), 2 * (q2 * q3 + P * q0 * q1), qbar + 2 * q3 ** 2]])
    return g

def qu2ax(q):
    return

def qu2ro(q):
    s = np.sqrt(q[1] ** 2 + q[2] ** 2 + q[3] ** 2)
    t = np.arctan(q[0])
    return t * np.array([q[1] / s, q[2] / s, q[3] / s])

In [6]:
conversions = [[None, eu2om, eu2ax, eu2ro, eu2qu],
               [om2eu, None, om2ax, om2ro, om2qu],
               [ax2eu, ax2om, None, ax2ro, ax2qu],
               [ro2eu, ro2om, ro2ax, None, ro2qu],
               [qu2eu, qu2om, qu2ax, qu2ro, None]]

def dist(a, b):
    # manhattan distance
    return np.max(np.abs(a - b))

def euler_dist(a, b):
    return dist(eu2qu(a), eu2qu(b))

def om_dist(a, b):
    return dist(a.flatten(), b.flatten())

NameError: name 'eu2om' is not defined

In [7]:
index = 20
print('input euler', eulers[index])

print('\n* eu2om and om2eu')
#eu2om(eulers[index])
print('output euler', om2eu(eu2om(eulers[index])))
print(dist(eulers[index], om2eu(eu2om(eulers[index]))))
print(euler_dist(eulers[index], om2eu(eu2om(eulers[index]))))

print('\n* eu2ax and ax2eu')
print(eu2ax(eulers[index]))
print('output euler', ax2eu(eu2ax(eulers[index])))
print(euler_dist(eulers[index], ax2eu(eu2ax(eulers[index]))))

print('\n* om2ax and ax2om')
print('input', eu2om(eulers[index]))
print('output', ax2om(om2ax(eu2om(eulers[index]))))
print(om_dist(eu2om(eulers[index]), ax2om(om2ax(eu2om(eulers[index])))))

print('\n* eu2ro and ro2eu')
print(eu2ro(eulers[index]))
print('output euler', ro2eu(eu2ro(eulers[index])))
print(euler_dist(eulers[index], ro2eu(eu2ro(eulers[index]))))
print('with ax repr to compute distance:')
print(eu2ax(eulers[index]))
print('output ax', ro2ax(eu2ro(eulers[index])))
print(dist(eu2ax(eulers[index]), ro2ax(eu2ro(eulers[index]))))

print('\n* ax2ro and ro2ax')
print('input ax', eu2ax(eulers[index]))
print('output ax', ro2ax(ax2ro(eu2ax(eulers[index]))))
print(dist(eu2ax(eulers[index]), ro2ax(ax2ro(eu2ax(eulers[index])))))

print('\n* om2ro and ro2om')
print('input', eu2om(eulers[index]))
print('output', ro2om(om2ro(eu2om(eulers[index]))))
print(om_dist(eu2om(eulers[index]), ro2om(om2ro(eu2om(eulers[index])))))

print('\n* eu2qu and qu2eu')
print(eu2qu(eulers[index]))
print('output euler', qu2eu(eu2qu(eulers[index])))
print(euler_dist(eulers[index], qu2eu(eu2qu(eulers[index]))))

print('\n* om2qu and qu2om')
print('input', eu2om(eulers[index]))
print('output', qu2om(om2qu(eu2om(eulers[index]))))
print(om_dist(eu2om(eulers[index]), qu2om(om2qu(eu2om(eulers[index])))))


input euler [1.00000000e-05 3.14167265e-01 2.82742539e+00]

* eu2om and om2eu


NameError: name 'eu2om' is not defined

In [8]:
for i, euler in enumerate(eulers):
    if euler_dist(euler, om2eu(eu2om(euler))) > 1e-6:
        print('error in conversion at index {} with angles {}'.format(i, euler))
    if euler_dist(euler, ax2eu(eu2ax(euler))) > 1e-6:
        print('error in conversion at index {} with angles {}'.format(i, euler))
    if om_dist(eu2om(euler), ax2om(om2ax(eu2om(euler)))) > 1e-6:
        print('error in conversion at index {} with angles {}'.format(i, euler))
    if dist(eu2ax(euler), ro2ax(eu2ro(euler))) > 1e-6:
        print('error in conversion at index {} with angles {}'.format(i, euler))
    if dist(eu2ax(euler), ro2ax(ax2ro(eu2ax(euler)))) > 1e-6:
        print('error in conversion at index {} with angles {}'.format(i, euler))
    if om_dist(eu2om(euler), ro2om(om2ro(eu2om(euler)))) > 1e-6:
        print('error in conversion at index {} with angles {}'.format(i, euler))
    if euler_dist(euler, qu2eu(eu2qu(euler))) > 1e-6:
        print('error in conversion at index {} with angles {}'.format(i, euler))
    #if dist(eu2qu(euler), eu2qu_check(euler)) > 1e-6:
    #    # fix pb with pi rotation
    #    print('error in conversion at index {} with angles {}'.format(i, euler))
    #if om_dist(qu2om(eu2qu(euler)), qu2om_check(eu2qu(euler))) > 1e-6:
    #    print('error in conversion at index {} with angles {}'.format(i, euler))
    if om_dist(eu2om(euler), qu2om(om2qu(eu2om(euler)))) > 1e-6:
        print('error in conversion at index {} with angles {}'.format(i, euler))

NameError: name 'euler_dist' is not defined

In [9]:
eu2om_vect(eulers)[500]

array([[-9.04506734e-01,  2.93890053e-01,  3.09024603e-01],
       [-3.09015092e-01, -9.51057134e-01,  1.89222995e-17],
       [ 2.93900053e-01, -9.54932662e-02,  9.51054044e-01]])